In [26]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers.core import Dense,Activation,Dropout
from keras.layers import LSTM
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from keras.callbacks import EarlyStopping
import math
import os
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer


In [ ]:
hotel_details=pd.read_csv('/content/Hotel_details.csv',delimiter=',')
hotel_rooms=pd.read_csv('/content/Hotel_Room_attributes.csv',delimiter=',')
hotel_cost=pd.read_csv('/content/hotels_RoomPrice.csv',delimiter=',')

In [ ]:
hotel_details.head()

,id,hotelid,hotelname,address,city,country,zipcode,propertytype,starrating,latitude,longitude,Source,url,curr
0,46406,1771651,Mediteran Bungalow Galeb,Vukovarska 7,Omis,Croatia,21310.0,Holiday parks,4,43.440124,16.682505,2,https://www.booking.com/hotel/hr/bungalow-luxu...,EUR
1,46407,177167,Hotel Polonia,Plac Teatralny 5,Torun,Poland,NaN,Hotels,3,53.012329,18.603800,5,https://www.agoda.com/en-gb/hotel-polonia/hote...,EUR
2,46408,1771675,Rifugio Sass Bece,"Belvedere del Pordoi,1",Canazei,Italy,38032.0,Hotels,3,46.477920,11.813350,2,http://www.booking.com/hotel/it/rifugio-sass-b...,EUR
3,46409,177168,Madalena Hotel,Mykonos,Mykonos,Greece,84600.0,Hotels,3,37.452316,25.329849,5,https://www.agoda.com/en-gb/madalena-hotel/hot...,EUR
4,46410,1771718,Pension Morenfeld,Mair im Korn Strasse 2,Lagundo,Italy,39022.0,Hotels,3,46.682780,11.131736,2,http://www.booking.com/hotel/it/pension-morenf...,EUR


In [ ]:
hotel_rooms.head()

,id,hotelcode,roomamenities,roomtype,ratedescription
0,50677497,634876,Air conditioning: ;Alarm clock: ;Carpeting: ;C...,Double Room,"Room size: 15 m²/161 ft², Shower, 1 king bed"
1,50672149,8328096,Air conditioning: ;Closet: ;Fireplace: ;Free W...,Vacation Home,"Shower, Kitchenette, 2 bedrooms, 1 double bed ..."
2,50643430,8323442,Air conditioning: ;Closet: ;Dishwasher: ;Firep...,Vacation Home,"Shower, Kitchenette, 2 bedrooms, 1 double bed ..."
3,50650317,7975,Air conditioning: ;Clothes rack: ;Coffee/tea m...,Standard Triple Room,"Room size: 20 m²/215 ft², Shower, 3 single beds"
4,50650318,7975,Air conditioning: ;Clothes rack: ;Coffee/tea m...,Standard Triple Room,"Room size: 20 m²/215 ft², Shower, 3 single beds"


In [ ]:
del hotel_details['id']
del hotel_rooms['id']
del hotel_details['zipcode']

In [ ]:
hotel_details=hotel_details.dropna()
hotel_rooms=hotel_rooms.dropna()

In [ ]:
hotel_details.drop_duplicates(subset='hotelid',keep=False,inplace=True)

In [ ]:
hotel=pd.merge(hotel_rooms,hotel_details,left_on='hotelcode',right_on='hotelid',how='inner')

In [ ]:
hotel.columns

Index(['hotelcode', 'roomamenities', 'roomtype', 'ratedescription', 'hotelid',
       'hotelname', 'address', 'city', 'country', 'propertytype', 'starrating',
       'latitude', 'longitude', 'Source', 'url', 'curr'],
      dtype='object')

In [ ]:
del hotel['hotelid']
del hotel['url']
del hotel['curr']
del hotel['Source']

In [ ]:
hotel.columns

Index(['hotelcode', 'roomamenities', 'roomtype', 'ratedescription',
       'hotelname', 'address', 'city', 'country', 'propertytype', 'starrating',
       'latitude', 'longitude'],
      dtype='object')

Recommender system based only on City and ratings about the hote

In [ ]:
def citybased(city):
    hotel['city']=hotel['city'].str.lower()
    citybase=hotel[hotel['city']==city.lower()]
    citybase=citybase.sort_values(by='starrating',ascending=False)
    citybase.drop_duplicates(subset='hotelcode',keep='first',inplace=True)
    if(citybase.empty==0):
        hname=citybase[['hotelname','starrating','address','roomamenities','ratedescription']]
        return hname.head()
    else:
        print('No Hotels Available')

In [ ]:
print('Top 5 hotels')
citybased(input())

Top 5 hotels
Paris


,hotelname,starrating,address,roomamenities,ratedescription
9662,Hotel Best Western Louvre Saint Honore,4,"141, rue Saint-Honore",Air conditioning: ;Alarm clock: ;Bathrobes: ;C...,"Room size: 24 m²/258 ft², Non-smoking, Separat..."
25030,Novotel Suites Paris Expo Pte Versailles,4,4 Boulevard Brune,Air conditioning: ;Alarm clock: ;Carpeting: ;C...,"Room size: 30 m²/323 ft², City view, Non-smoki..."
9982,Mercure Paris Roissy Charles de Gaulle Hotel,4,3 Allee du Verger,Air conditioning: ;Alarm clock: ;Bathrobes: ;C...,"Room size: 47 m²/506 ft², Non-smoking, Shower ..."
28149,Royal Hotel,4,33 avenue de Friedland,Air conditioning: ;Blackout curtains: ;Carpeti...,"Room size: 23 m²/248 ft², Street view, Non-smo..."
11162,Hotel Le Chaplain Rive Gauche,4,11 Bis Rue Jules Chaplain,Air conditioning: ;Closet: ;Coffee/tea maker: ...,"Room size: 16 m²/172 ft², Street view, Non-smo..."


In [ ]:
room_no=[
     ('king',2),
   ('queen',2), 
    ('triple',3),
    ('master',3),
   ('family',4),
   ('murphy',2),
   ('quad',4),
   ('double-double',4),
   ('mini',2),
   ('studio',1),
    ('junior',2),
   ('apartment',4),
    ('double',2),
   ('twin',2),
   ('double-twin',4),
   ('single',1),
     ('diabled',1),
   ('accessible',1),
    ('suite',2),
    ('one',2)
   ]

In [ ]:
def calc():
    guests_no=[]
    for i in range(hotel.shape[0]):
        temp=hotel['roomtype'][i].lower().split()
        flag=0
        for j in range(len(temp)):
            for k in range(len(room_no)):
                if temp[j]==room_no[k][0]:
                    guests_no.append(room_no[k][1])
                    flag=1
                    break
            if flag==1:
                break
        if flag==0:
            guests_no.append(2)
    hotel['guests_no']=guests_no

calc()

In [ ]:
def pop_citybased(city,number):
    hotel['city']=hotel['city'].str.lower()
    popbased=hotel[hotel['city']==city.lower()]
    popbased=popbased[popbased['guests_no']==number].sort_values(by='starrating',ascending=False)
    popbased.drop_duplicates(subset='hotelcode',keep='first',inplace=True)
    if popbased.empty==True:
        print('Sorry No Hotels Available\n tune your constraints')
    else:
        return popbased[['hotelname','roomtype','guests_no','starrating','address','roomamenities','ratedescription']].head(10)

In [ ]:
pop_citybased(input(),(int(input())))

London
4


,hotelname,roomtype,guests_no,starrating,address,roomamenities,ratedescription
925,Holiday Inn London - Heathrow T5,Family Room,4,4,Old Bath Road,Air conditioning: ;Alarm clock: ;Blackout curt...,"Room size: 25 m²/269 ft², Non-smoking, Separat..."
27679,Hilton London Metropole Hotel,Family Plus Superior Room,4,4,225 Edgware Road,Air conditioning: ;Free Wi-Fi in all rooms!: ;...,"Room size: 25 m²/269 ft², Non-smoking, 4 queen..."
24212,Mornington Hotel London Victoria,Deluxe Family Suite,4,4,25 Gloucester Street,Additional toilet: ;Air conditioning: ;Alarm c...,"Room size: 35 m²/377 ft², Non-smoking, 3 bathr..."
25849,Innkeeper's Lodge Old Windsor,Family Room (2 Adults + 1 Child),4,4,"Old Windsor Toby Carvery, Straight Road",Air conditioning: ;Carpeting: ;Closet: ;Clothe...,"Room size: 18 m²/194 ft², Shower, 1 double bed..."
30868,The Colonnade London Hotel,Three-Bedroom Luxury Apartment,4,4,2 Warrington Crescent,Additional bathroom: ;Additional toilet: ;Air ...,"Room size: 121 m²/1302 ft², 2 bathrooms, Showe..."
32958,Moxy London Heathrow Airport,"Moxyfied Family Sleeper, Larger Guest room",4,4,804-834 Bath Road,Air conditioning: ;Carpeting: ;Closet: ;Clothe...,"Room size: 34 m²/366 ft², City view, Non-smoki..."
33603,Mercure London Hyde Park Hotel,Family Room,4,4,8-14 Talbot Square,Air conditioning: ;Desk: ;Free Wi-Fi in all ro...,"Room size: 31 m²/334 ft², Non-smoking, 1 singl..."
35182,Acorn Lodge Gatwick,Family En-Suite Room,4,4,"79 Massetts Road, Horley",Air conditioning: ;Carpeting: ;Closet: ;Coffee...,"Shower, 1 double bed and 1 bunk bed"
30633,Courtyard Luton Airport,"Superior Family Room, Guest room, 1 King, Sofa...",4,4,Airport Way,Additional toilet: ;Air conditioning: ;Alarm c...,"Shower, 1 king bed and 1 sofa bed"
8374,Georgian House Hotel,Family Suite,4,4,37-39 St Georges Drive,Air conditioning: ;Closet: ;Coffee/tea maker: ...,"Room size: 32 m²/344 ft², Non-smoking, Shower,..."


In [ ]:
hotel['roomamenities']=hotel['roomamenities'].str.replace(': ;',',')

In [22]:
def requirementbased(city,number,features):
    hotel['city']=hotel['city'].str.lower()
    hotel['roomamenities']=hotel['roomamenities'].str.lower()
    features=features.lower()
    features_tokens=word_tokenize(features)  
    sw = stopwords.words('english')
    lemm = WordNetLemmatizer()
    f1_set = {w for w in features_tokens if not w in sw}
    f_set=set()
    for se in f1_set:
        f_set.add(lemm.lemmatize(se))
    reqbased=hotel[hotel['city']==city.lower()]
    reqbased=reqbased[reqbased['guests_no']==number]
    reqbased=reqbased.set_index(np.arange(reqbased.shape[0]))
    l1 =[];l2 =[];cos=[];
    #print(reqbased['roomamenities'])
    for i in range(reqbased.shape[0]):
        temp_tokens=word_tokenize(reqbased['roomamenities'][i])
        temp1_set={w for w in temp_tokens if not w in sw}
        temp_set=set()
        for se in temp1_set:
            temp_set.add(lemm.lemmatize(se))
        rvector = temp_set.intersection(f_set)
        #print(rvector)
        cos.append(len(rvector))
    reqbased['similarity']=cos
    reqbased=reqbased.sort_values(by='similarity',ascending=False)
    reqbased.drop_duplicates(subset='hotelcode',keep='first',inplace=True)
    return reqbased[['hotelname','roomtype','guests_no','starrating','address','roomamenities','ratedescription','similarity']].head(10)

Requirement And special needs based Recommender

In [38]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [40]:
requirementbased('London',1,'I need a extra toilet and room should be completely air conditioned.I should have a bathrobe.')

,hotelname,roomtype,guests_no,starrating,address,roomamenities,ratedescription,similarity
231,The Colonnade London Hotel,Studio,1,4,2 Warrington Crescent,"additional toilet,air conditioning,alarm clock...","Room size: 28 m²/301 ft², Balcony/terrace, Non...",4
208,Best Western Palm Hotel,Standard Room With Three Single Beds,1,3,64-76 Hendon Way,"additional toilet,air conditioning,carpeting,c...","Room size: 28 m²/301 ft², Bathtub, 3 single beds",3
61,Little Foxes Hotel,Single - 1 Person,1,3,"Charlwood Road, Crawley, West sussex","additional toilet,air conditioning,alarm clock...","Room size: 15 m²/161 ft², Country/region view,...",3
131,Megaro Hotel,Corner Studio,1,4,Belgrove Street,"air conditioning,bathrobes,carpeting,closet,cl...","Room size: 35 m²/377 ft², Balcony/terrace, Sho...",3
63,Belgrave Hotel Oval,Single Room,1,4,9-13 Clapham Road,"additional toilet,air conditioning,carpeting,c...","Room size: 10 m²/108 ft², Non-smoking, Shower,...",3
72,The Ship Inn,Single Room with Shared Bathroom,1,3,290 Westferry Road,"air conditioning,bathrobes,carpeting,closet,fr...","Shared bathroom, Kitchenette, 1 single bed",3
248,Park International Hotel,Standard Single Room,1,4,117-129 Cromwell Road,"additional toilet,air conditioning,blackout cu...","Room size: 11 m²/118 ft², Non-smoking, Shower,...",3
11,The Springfield Hotel,Single Room,1,2,154 Sussex Gardens,"additional toilet,air conditioning,alarm clock...","Street view, Shower, 1 single bed",3
308,South Point Suites - London Bridge,Deluxe Studio Suite,1,4,Bermondsey Street,"additional bathroom,additional toilet,air cond...","Room size: 45 m²/484 ft², 2 bathrooms, Shower,...",3
184,Crowne Plaza Gerrards Cross,1 Bedroom Accessible Roll In Shower Non-Smoking,1,4,"Oxford Road, Gerrards Cross","air conditioning,bathrobes,closet,free wi-fi i...","Room size: 25 m²/269 ft², Non-smoking, Bathtub",3


In [39]:
requirementbased('Paris',1,'I need a extra toilet and room should be completely air conditioned.I should have a bathrobe and an alarm clock')

,hotelname,roomtype,guests_no,starrating,address,roomamenities,ratedescription,similarity
61,Best Western Premier Amiral Hotel,Amiral - Single,1,4,98 Avenue d Italie,"air conditioning,alarm clock,bathrobes,blackou...","Room size: 14 m²/151 ft², Non-smoking, Shower ...",5
52,Mercure Paris 19 Philharmonie,Classic 2 Single Beds,1,4,"216, Avenue Jean Jaures","air conditioning,alarm clock,bathrobes,carpeti...","Room size: 30 m²/323 ft², City view, Shower an...",5
24,Hotel Royal Madeleine,Single Room,1,4,29 rue de l'Arcade,"air conditioning,alarm clock,bathrobes,closet,...","Room size: 13 m²/140 ft², City view, Non-smoki...",5
143,Hotel Malte - Astotel,Standard Single,1,4,"63, Rue de Richelieu","air conditioning,alarm clock,bathrobes,blackou...","Room size: 15 m²/161 ft², Non-smoking, Shower,...",5
77,Hotel du Bois,Classic Single Room,1,3,11 rue du dome 75116 PARIS,"air conditioning,alarm clock,carpeting,closet,...","Room size: 13 m²/140 ft², Shower, 1 single bed",4
48,Hotel Savoy,Single,1,3,20 Rue Villeneuve,"air conditioning,alarm clock,closet,clothes ra...","Room size: 11 m²/118 ft², Non-smoking, Shower ...",4
80,Hotel Sunny,Single Bed for 1 person,1,2,48 boulevard port royal,"air conditioning,alarm clock,desk,free wi-fi i...","Room size: 10 m²/108 ft², Courtyard view, Non-...",4
41,Apogia Hotel,Confort Single room,1,3,"14, Boulevard Paul Vaillant Couturier","air conditioning,alarm clock,carpeting,closet,...","Room size: 22 m²/237 ft², Shower and bathtub, ...",4
66,MidNight Hotel Paris,Classic Single Room,1,3,133 rue du Faubourg St-Denis,"air conditioning,alarm clock,closet,desk,dress...","Room size: 14 m²/151 ft², Street view, Shower,...",4
112,Sweet Hotel,Single Comfort,1,3,15 rue Gossec,"air conditioning,alarm clock,blackout curtains...","Room size: 15 m²/161 ft², Non-smoking, Shower,...",4
